In [ ]:
branch? 

Twitter Fingers == Trigger Fingers: A Look at Gun Violence

We have to first extract the sources from complete_project_dataset, then get the text from it

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
#reads in complete project data
gunviolencedataset = pd.read_csv("complete_project_dataset")
sourceurl = gunviolencedataset["Source"]

#sourceurl[15]

The function that takes the source url, goes to the url, reads in the text from that site. 

In [2]:
def getsourcetext(urlseries):
    import urllib.request
    from bs4 import BeautifulSoup
    #initialize beautifulsoup
    soup = BeautifulSoup('''<html> empty </html>''', 'html.parser')
    k = len(urlseries)
    #textfile = open("all text", "w+",  encoding="utf-8")
    counter = 0
    for k in urlseries:
        print(counter)
        counter = counter + 1
        # here we actually access the website
        try:
            print("opening link")
            html = urllib.request.urlopen(k, [60])
            print("reading an open link")
            html = html.read()
            print("decoding an open link")
            htmlfile = html.decode('utf-8')
        #need to make exceptions for 404
        except Exception as e:
            print(str(e))
        else:
            print("made it past try block")
            singlesoup = BeautifulSoup(htmlfile, 'html.parser')
            text = singlesoup
            #textfile.write(text)
            soup.append(singlesoup)
    #textfile.close()
    return soup

In [ ]:
rawtext = getsourcetext(sourceurl) #as a beautiful soup object



0
opening link
HTTP Error 404: Not Found
1
opening link
reading an open link
decoding an open link
made it past try block
2
opening link


In [ ]:
print(soup[0])
#add a counter to above just for proof of life